In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from catboost import CatBoostClassifier
import xgboost as xgb
import random

In [2]:
data = pd.read_csv("minute_data/BTC-USD_1M_SIGNALS.csv")

In [3]:
data

,Unix,Date,Symbol,Open,High,Low,Close,Volume USD,Variation,RSI,MACD,MACD_H,-DM,+DM,ADX14
0,1483228860,2017-01-01 00:01:00,BTC/USD,966.34,966.34,966.34,966.34,-0.312459,-0.006352,-0.291721,0.003268,0.001095,-0.296745,-0.163178,-0.618641
1,1483228920,2017-01-01 00:02:00,BTC/USD,966.16,966.37,966.16,966.37,-0.310186,-0.000855,0.828292,0.003534,0.002246,-0.285351,-0.163178,-0.668323
2,1483228980,2017-01-01 00:03:00,BTC/USD,966.37,966.37,966.37,966.37,-0.348678,-0.000855,0.828292,0.003738,0.003484,-0.296745,-0.163178,-0.647506
3,1483229040,2017-01-01 00:04:00,BTC/USD,966.37,966.37,966.37,966.37,-0.348678,-0.011849,0.835669,0.003932,0.004996,-0.293490,-0.163178,-0.649919
4,1483229100,2017-01-01 00:05:00,BTC/USD,966.43,966.43,966.43,966.43,-0.348169,-0.028336,0.910591,0.004110,0.006831,-0.288606,-0.163178,-0.652332
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2674991,1646105880,2022-03-01 03:38:00,BTC/USD,43077.82,43077.82,43049.46,43049.46,-0.343969,0.205143,3.243746,202.140130,315.950658,-0.296745,1.189788,0.570316
2674992,1646105940,2022-03-01 03:39:00,BTC/USD,43035.16,43035.16,42999.44,42999.44,-0.174815,-0.069151,3.243036,182.098937,328.738652,0.603408,-0.163178,0.176675
2674993,1646106000,2022-03-01 03:40:00,BTC/USD,43022.24,43022.24,43016.03,43016.03,-0.346723,-0.126531,3.243035,153.986631,318.063305,-0.177376,-0.163178,-0.189747
2674994,1646106060,2022-03-01 03:41:00,BTC/USD,43018.23,43046.59,43018.23,43046.58,-0.318365,-0.000855,3.243946,115.826724,271.530088,1.241492,-0.163178,-0.539146


In [4]:
data[0:20]

,Unix,Date,Symbol,Open,High,Low,Close,Volume USD,Variation,RSI,MACD,MACD_H,-DM,+DM,ADX14
0,1483228860,2017-01-01 00:01:00,BTC/USD,966.34,966.34,966.34,966.34,-0.312459,-0.006352,-0.291721,0.003268,0.001095,-0.296745,-0.163178,-0.618641
1,1483228920,2017-01-01 00:02:00,BTC/USD,966.16,966.37,966.16,966.37,-0.310186,-0.000855,0.828292,0.003534,0.002246,-0.285351,-0.163178,-0.668323
2,1483228980,2017-01-01 00:03:00,BTC/USD,966.37,966.37,966.37,966.37,-0.348678,-0.000855,0.828292,0.003738,0.003484,-0.296745,-0.163178,-0.647506
3,1483229040,2017-01-01 00:04:00,BTC/USD,966.37,966.37,966.37,966.37,-0.348678,-0.011849,0.835669,0.003932,0.004996,-0.293490,-0.163178,-0.649919
4,1483229100,2017-01-01 00:05:00,BTC/USD,966.43,966.43,966.43,966.43,-0.348169,-0.028336,0.910591,0.004110,0.006831,-0.288606,-0.163178,-0.652332
5,1483229160,2017-01-01 00:06:00,BTC/USD,966.58,966.58,966.58,966.58,-0.347099,-0.000855,1.110668,0.004124,0.008586,-0.296745,-0.163178,-0.665621
6,1483229220,2017-01-01 00:07:00,BTC/USD,966.58,966.58,966.58,966.58,-0.348678,-0.000855,1.110668,0.003720,0.009398,-0.296745,-0.163178,-0.721027
7,1483229280,2017-01-01 00:08:00,BTC/USD,966.58,966.58,966.58,966.58,-0.348678,-0.000855,0.063198,0.003137,0.009823,-0.296745,-0.163178,-0.740746
8,1483229340,2017-01-01 00:09:00,BTC/USD,966.58,966.58,966.58,966.58,-0.348678,0.000977,0.063198,0.002335,0.009628,-0.296745,-0.162861,-0.762462
9,1483229400,2017-01-01 00:10:00,BTC/USD,966.57,966.57,966.57,966.57,-0.345367,0.022965,0.987989,0.001263,0.008497,-0.296745,-0.163178,-0.786936


In [5]:
predict_data = data.copy().drop(["Open","Close","High","Low", "Symbol"],axis=1)
#max_days = 5
#target_range = 2
max_days = 30
target_range = 10
for i in range(1,max_days):#2jours
    #predict_data[["Variation-{}".format(i),"Vol-{}".format(i),"RSI-{}".format(i),"MACD-{}".format(i),"MACD_H-{}".format(i),"CONF-{}".format(i),"TRANS-{}".format(i),"REV-{}".format(i),"FnG-{}".format(i)]] = data[["Variation","Volume","RSI","MACD","MACD_H","Confirmation Time","Transactions","Miners Revenue","FnG"]].shift(i)
    #predict_data[["Variation-{}".format(i),"Vol-{}".format(i),"RSI-{}".format(i),"MACD-{}".format(i),"MACD_H-{}".format(i),"CONF-{}".format(i),"TRANS-{}".format(i),"REV-{}".format(i),"FnG-{}".format(i), "ADX-{}".format(i), "+DM-{}".format(i), "-DM-{}".format(i)]] = data[["Variation","Volume","RSI","MACD","MACD_H","Confirmation Time","Transactions","Miners Revenue","FnG","ADX14","+DM","-DM"]].shift(i)
    predict_data[["Variation-{}".format(i),"Vol-{}".format(i),"RSI-{}".format(i),"MACD-{}".format(i),"MACD_H-{}".format(i)]] = data[["Variation","Volume USD","RSI","MACD","MACD_H"]].shift(i)
#predict_data["Target"] = (data["Variation"].shift(-1) >= 0)
predict_data["Target"] = (data["Close"].shift(-target_range) - data["Close"] >= 0)
predict_data["Target"] = np.where(predict_data["Target"] == True, 1, 0)
predict_data.dropna(inplace=True)
predict_data.reset_index(inplace=True,drop=True)
predict_data = predict_data[0:len(predict_data)-target_range]

c:\Users\Alexis\AppData\Local\Programs\Python\Python38\lib\site-packages\pandas\core\frame.py:3636: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self[k1] = value[k2]
<ipython-input-5-fcd053aba5c6>:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  predict_data["Target"] = (data["Close"].shift(-target_range) - data["Close"] >= 0)


In [6]:
predict_data

,Unix,Date,Volume USD,Variation,RSI,MACD,MACD_H,-DM,+DM,ADX14,Variation-1,Vol-1,RSI-1,MACD-1,MACD_H-1,Variation-2,Vol-2,RSI-2,MACD-2,MACD_H-2,Variation-3,Vol-3,RSI-3,MACD-3,MACD_H-3,Variation-4,Vol-4,RSI-4,MACD-4,MACD_H-4,Variation-5,Vol-5,RSI-5,MACD-5,MACD_H-5,Variation-6,Vol-6,RSI-6,MACD-6,MACD_H-6,...,Vol-22,RSI-22,MACD-22,MACD_H-22,Variation-23,Vol-23,RSI-23,MACD-23,MACD_H-23,Variation-24,Vol-24,RSI-24,MACD-24,MACD_H-24,Variation-25,Vol-25,RSI-25,MACD-25,MACD_H-25,Variation-26,Vol-26,RSI-26,MACD-26,MACD_H-26,Variation-27,Vol-27,RSI-27,MACD-27,MACD_H-27,Variation-28,Vol-28,RSI-28,MACD-28,MACD_H-28,Variation-29,Vol-29,RSI-29,MACD-29,MACD_H-29,Target
0,1483230600,2017-01-01 00:30:00,-0.348678,-0.000855,-3.026184,-0.005897,-0.013714,-0.296745,-0.163178,3.826386,-0.000855,-0.315971,-3.078899,-0.006453,-0.012442,-0.068736,-0.348678,-3.104569,-0.007500,-0.012720,-0.000855,-0.348678,-3.145534,-0.008245,-0.012144,-0.000855,-0.348678,-3.145534,-0.008745,-0.011036,-0.000855,-0.348678,-3.145265,-0.009047,-0.009628,-0.000855,-0.346488,-3.145265,-0.009191,-0.008083,...,-0.348678,0.063198,0.003137,0.009823,-0.000855,-0.348678,1.110668,0.003720,0.009398,-0.000855,-0.347099,1.110668,0.004124,0.008586,-0.028336,-0.348169,0.910591,0.004110,0.006831,-0.011849,-0.348678,0.835669,0.003932,0.004996,-0.000855,-0.348678,0.828292,0.003738,0.003484,-0.000855,-0.310186,0.828292,0.003534,0.002246,-0.006352,-0.312459,-0.291721,0.003268,0.001095,1
1,1483230660,2017-01-01 00:31:00,-0.348678,-0.000855,-1.342597,-0.005114,-0.014555,-0.296745,-0.163178,3.660364,-0.000855,-0.348678,-3.026184,-0.005897,-0.013714,-0.000855,-0.315971,-3.078899,-0.006453,-0.012442,-0.068736,-0.348678,-3.104569,-0.007500,-0.012720,-0.000855,-0.348678,-3.145534,-0.008245,-0.012144,-0.000855,-0.348678,-3.145534,-0.008745,-0.011036,-0.000855,-0.348678,-3.145265,-0.009047,-0.009628,...,-0.348678,0.063198,0.002335,0.009628,-0.000855,-0.348678,0.063198,0.003137,0.009823,-0.000855,-0.348678,1.110668,0.003720,0.009398,-0.000855,-0.347099,1.110668,0.004124,0.008586,-0.028336,-0.348169,0.910591,0.004110,0.006831,-0.011849,-0.348678,0.835669,0.003932,0.004996,-0.000855,-0.348678,0.828292,0.003738,0.003484,-0.000855,-0.310186,0.828292,0.003534,0.002246,1
2,1483230720,2017-01-01 00:32:00,-0.348678,-0.000855,-1.407712,-0.004053,-0.014689,-0.296745,-0.163178,3.494341,-0.000855,-0.348678,-1.342597,-0.005114,-0.014555,-0.000855,-0.348678,-3.026184,-0.005897,-0.013714,-0.000855,-0.315971,-3.078899,-0.006453,-0.012442,-0.068736,-0.348678,-3.104569,-0.007500,-0.012720,-0.000855,-0.348678,-3.145534,-0.008245,-0.012144,-0.000855,-0.348678,-3.145534,-0.008745,-0.011036,...,-0.345367,0.987989,0.001263,0.008497,0.000977,-0.348678,0.063198,0.002335,0.009628,-0.000855,-0.348678,0.063198,0.003137,0.009823,-0.000855,-0.348678,1.110668,0.003720,0.009398,-0.000855,-0.347099,1.110668,0.004124,0.008586,-0.028336,-0.348169,0.910591,0.004110,0.006831,-0.011849,-0.348678,0.835669,0.003932,0.004996,-0.000855,-0.348678,0.828292,0.003738,0.003484,1
3,1483230780,2017-01-01 00:33:00,-0.348678,-0.000855,-1.746586,-0.002653,-0.013737,-0.296745,-0.163178,3.321486,-0.000855,-0.348678,-1.407712,-0.004053,-0.014689,-0.000855,-0.348678,-1.342597,-0.005114,-0.014555,-0.000855,-0.348678,-3.026184,-0.005897,-0.013714,-0.000855,-0.315971,-3.078899,-0.006453,-0.012442,-0.068736,-0.348678,-3.104569,-0.007500,-0.012720,-0.000855,-0.348678,-3.145534,-0.008245,-0.012144,...,-0.342565,0.935494,-0.000112,0.006078,0.022965,-0.345367,0.987989,0.001263,0.008497,0.000977,-0.348678,0.063198,0.002335,0.009628,-0.000855,-0.348678,0.063198,0.003137,0.009823,-0.000855,-0.348678,1.110668,0.003720,0.009398,-0.000855,-0.347099,1.110668,0.004124,0.008586,-0.028336,-0.348169,0.910591,0.004110,0.006831,-0.011849,-0.348678,0.835669,0.003932,0.004996,1
4,1483230840,2017-01-01 00:34:00,-0.348678,-0.207924,-2.053923,-0.000840,-0.011184,-0.235433,-0.163178,3.148631,-0.000855,-0.348678,-1.746586,-0.002653,-0.013737,-0.000855,-0.348678,-1.407712,-0.004053,-0.

In [7]:
predict_data = predict_data[[i % int(max_days) == 0 for i in range(len(predict_data))]]

In [8]:
predict_data

,Unix,Date,Volume USD,Variation,RSI,MACD,MACD_H,-DM,+DM,ADX14,Variation-1,Vol-1,RSI-1,MACD-1,MACD_H-1,Variation-2,Vol-2,RSI-2,MACD-2,MACD_H-2,Variation-3,Vol-3,RSI-3,MACD-3,MACD_H-3,Variation-4,Vol-4,RSI-4,MACD-4,MACD_H-4,Variation-5,Vol-5,RSI-5,MACD-5,MACD_H-5,Variation-6,Vol-6,RSI-6,MACD-6,MACD_H-6,...,Vol-22,RSI-22,MACD-22,MACD_H-22,Variation-23,Vol-23,RSI-23,MACD-23,MACD_H-23,Variation-24,Vol-24,RSI-24,MACD-24,MACD_H-24,Variation-25,Vol-25,RSI-25,MACD-25,MACD_H-25,Variation-26,Vol-26,RSI-26,MACD-26,MACD_H-26,Variation-27,Vol-27,RSI-27,MACD-27,MACD_H-27,Variation-28,Vol-28,RSI-28,MACD-28,MACD_H-28,Variation-29,Vol-29,RSI-29,MACD-29,MACD_H-29,Target
0,1483230600,2017-01-01 00:30:00,-0.348678,-0.000855,-3.026184,-0.005897,-0.013714,-0.296745,-0.163178,3.826386,-0.000855,-0.315971,-3.078899,-0.006453,-0.012442,-0.068736,-0.348678,-3.104569,-0.007500,-0.012720,-0.000855,-0.348678,-3.145534,-0.008245,-0.012144,-0.000855,-0.348678,-3.145534,-0.008745,-0.011036,-0.000855,-0.348678,-3.145265,-0.009047,-0.009628,-0.000855,-0.346488,-3.145265,-0.009191,-0.008083,...,-0.348678,0.063198,0.003137,0.009823,-0.000855,-0.348678,1.110668,0.003720,0.009398,-0.000855,-0.347099,1.110668,0.004124,0.008586,-0.028336,-0.348169,0.910591,0.004110,0.006831,-0.011849,-0.348678,0.835669,0.003932,0.004996,-0.000855,-0.348678,0.828292,0.003738,0.003484,-0.000855,-0.310186,0.828292,0.003534,0.002246,-0.006352,-0.312459,-0.291721,0.003268,0.001095,1
30,1483232400,2017-01-01 01:00:00,-0.348121,-0.000855,3.231226,0.017293,0.011716,-0.296745,-0.163178,0.183850,-0.000855,-0.348678,3.225409,0.017017,0.008644,-0.000855,-0.348388,3.225409,0.016597,0.005804,0.070594,-0.348678,3.226101,0.016778,0.005123,-0.000855,-0.347980,3.225686,0.016719,0.003942,-0.004518,-0.348508,3.148116,0.016436,0.002404,-0.000855,-0.348512,2.798183,0.016016,0.000813,...,-0.346876,-1.212909,0.002404,-0.008493,0.000977,-0.324070,-1.183371,0.002094,-0.007614,-0.006352,-0.348678,-1.222406,0.001762,-0.006968,-0.000855,-0.347903,-1.222406,0.001472,-0.006342,-0.207924,-0.348678,-2.053923,-0.000840,-0.011184,-0.000855,-0.348678,-1.746586,-0.002653,-0.013737,-0.000855,-0.348678,-1.407712,-0.004053,-0.014689,-0.000855,-0.348678,-1.342597,-0.005114,-0.014555,0
60,1483234200,2017-01-01 01:30:00,-0.348632,-0.105622,-0.208651,-0.003674,-0.003934,-0.265818,-0.163178,-0.478428,-0.000855,-0.348678,-0.208651,-0.004664,-0.005764,-0.000855,-0.348678,-0.208651,-0.005397,-0.006548,-0.000855,-0.348604,-2.749448,-0.005919,-0.006620,0.195930,-0.348678,-0.221973,-0.004323,-0.001078,-0.000855,-0.348678,-0.100549,-0.003034,0.002545,-0.000855,-0.348037,-0.100549,-0.001999,0.004770,...,-0.305793,0.712666,0.006572,0.017188,-0.000855,-0.348678,1.412501,0.010090,0.023048,-0.000855,-0.348678,1.412501,0.012722,0.025392,-0.000855,-0.348250,1.029192,0.014628,0.025351,-0.000855,-0.348459,0.997911,0.015945,0.023760,-0.002687,-0.347933,0.802704,0.016766,0.021179,0.000977,-0.348678,0.806421,0.017227,0.018159,-0.000855,-0.348678,3.231226,0.017381,0.014935,1
90,1483236000,2017-01-01 02:00:00,-0.348465,-0.000855,1.343244,0.000077,-0.000869,-0.296745,-0.163178,-0.511264,-0.088998,-0.348678,-2.213954,-0.000880,-0.003224,-0.000855,-0.348678,-2.213954,-0.001630,-0.004560,-0.000855,-0.348678,-3.187031,-0.002209,-0.005179,-0.000855,-0.329393,-3.187031,-0.002648,-0.005303,0.057936,-0.348536,-1.019026,-0.002389,-0.003558,-0.098196,-0.348678,-1.638522,-0.003134,-0.004816,...,-0.348678,0.701775,-0.002053,0.008605,-0.000855,-0.310693,0.830637,-0.001325,0.008808,0.017514,-0.348507,0.957053,-0.000567,0.009048,-0.021059,-0.348441,0.935609,-0.000178,0.008267,-0.107395,-0.348678,-0.087014,-0.000937,0.004607,-0.000855,-0.348128,0.668297,-0.001532,0.002114,-0.002693,-0.348678,0.648824,-0.002009,0.000431,-0.000855,-0.347729,0.648824,-0.002370,-0.000609,1
120,1483237800,2017-01-01 02:30:00,-0.347934,-0.000855,3.014534,0.005766,0.012392,-0.296745,-0.133366,-0.308805,0.046866,-0.347204,2.845051,0.006730,0.012459,-0.022874,-0.348678,2.392193,0.007179,0.0

In [9]:
#corr = predict_data.corr("pearson")
#corr[["Target"]].to_clipboard()
#corr[["Target"]]

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
split_mode = "RAND"

if split_mode == "RAND":
    train_data, test_data, train_labels, test_labels = train_test_split(predict_data.drop(["Date", "Unix", "Target"],axis=1), predict_data["Target"], test_size=0.2, random_state=100)
elif split_mode == "STATIC":
    train_data = predict_data[0:int(0.66*len(predict_data))].drop(["Date", "Unix", "Target"],axis=1)
    train_labels = predict_data[0:int(0.66*len(predict_data))]["Target"]
    test_data = predict_data[int(0.66*len(predict_data)):len(predict_data)].drop(["Date", "Unix", "Target"],axis=1)
    test_labels = predict_data[int(0.66*len(predict_data)):len(predict_data)]["Target"]

In [12]:
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

In [65]:
algo = "RF"

if algo == "MLP":
    model = MLPClassifier(hidden_layer_sizes=(100,200,100),verbose=True)
elif algo == "RF":
    model = RandomForestClassifier(n_estimators=100,verbose=True)
elif algo == "CAT":
    model = CatBoostClassifier(iterations=100,depth=12)
elif algo == "TREE":
    model = DecisionTreeClassifier(splitter="random")
model.fit(train_data,train_labels)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   30.8s finished


RandomForestClassifier(verbose=True)

In [66]:
model.score(test_data,test_labels)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.4s finished


0.8734768081761006

In [67]:
model.score(train_data,train_labels)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.8s finished


1.0

In [68]:
from sklearn.metrics import confusion_matrix

In [69]:
preds = model.predict(test_data)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.4s finished


In [70]:
conf = confusion_matrix(test_labels,preds)
conf

array([[8923, 1235],
       [1340, 8854]], dtype=int64)

In [72]:
TN = conf[0,0] / (conf[0,0] + conf[1,0])
TN

0.8694338887264932

In [73]:
TP = conf[1,1] / (conf[1,1] + conf[0,1])
TP

0.8775894538606404

## Cross Validation

In [19]:
def cross_validation_loop(predict_data, model_name):
    scores = []
    TPs = []
    TNs = []
    for randomize in range(1,20):
        train_data, test_data, train_labels, test_labels = train_test_split(predict_data.drop(["Date", "Unix", "Target"],axis=1), predict_data["Target"], test_size=0.33, random_state=randomize)
        if model_name == "MLP":
            model = MLPClassifier(hidden_layer_sizes=(200,2000,200))
        elif model_name == "CAT":
            model = CatBoostClassifier(iterations=100,depth=12,verbose=False)
        elif model_name == "RF":
            model = RandomForestClassifier(n_estimators=100,verbose=False)
        model.fit(train_data, train_labels)
        scores.append(model.score(test_data, test_labels))
        conf = confusion_matrix(test_labels,model.predict(test_data))
        TN = conf[0,0] / (conf[0,0] + conf[1,0])
        TP = conf[1,1] / (conf[1,1] + conf[0,1])
        TPs.append(TP)
        TNs.append(TN)
        print("Score : {} || TP : {} || TN : {}".format(scores[-1],TP,TN))


    return scores,TPs,TNs

In [20]:
#scores,TPs,TNs = cross_validation_loop(predict_data,"CAT")

In [24]:
from statistics import mean

In [50]:
print("CatBoost score moyen : {} || TP moyen : {} || TN moyen : {}".format(mean(scores),mean(TPs),mean(TNs)))

score moyen : 0.5067452023560707 || TP moyen : 0.5411620348700146 || TN moyen : 0.4619500842660272


In [22]:
scores,TPs,TNs = cross_validation_loop(predict_data,"RF")

Score : 0.51985559566787 || TP : 0.5694444444444444 || TN : 0.46616541353383456
Score : 0.49458483754512633 || TP : 0.5194805194805194 || TN : 0.4634146341463415
Score : 0.5054151624548736 || TP : 0.5465116279069767 || TN : 0.4380952380952381
Score : 0.48375451263537905 || TP : 0.5136986301369864 || TN : 0.45038167938931295
Score : 0.4693140794223827 || TP : 0.4722222222222222 || TN : 0.4639175257731959
Score : 0.48736462093862815 || TP : 0.525 || TN : 0.4358974358974359
Score : 0.51985559566787 || TP : 0.5280898876404494 || TN : 0.5050505050505051
Score : 0.49458483754512633 || TP : 0.5611510791366906 || TN : 0.427536231884058
Score : 0.49458483754512633 || TP : 0.5384615384615384 || TN : 0.4380165289256198
Score : 0.4657039711191336 || TP : 0.4899328859060403 || TN : 0.4375
Score : 0.4693140794223827 || TP : 0.5205479452054794 || TN : 0.4122137404580153
Score : 0.5306859205776173 || TP : 0.5714285714285714 || TN : 0.46788990825688076
Score : 0.4584837545126354 || TP : 0.5 || TN : 0.4

In [25]:
print("Random Forest score moyen : {} || TP moyen : {} || TN moyen : {}".format(mean(scores),mean(TPs),mean(TNs)))

Random Forest score moyen : 0.5040851225536767 || TP moyen : 0.538149033546309 || TN moyen : 0.4600308310035195


In [26]:
scores,TPs,TNs = cross_validation_loop(predict_data,"MLP")

Score : 0.51985559566787 || TP : 0.5675675675675675 || TN : 0.46511627906976744
Score : 0.48014440433212996 || TP : 0.5068493150684932 || TN : 0.45038167938931295
Score : 0.5126353790613718 || TP : 0.56 || TN : 0.4566929133858268
Score : 0.5234657039711191 || TP : 0.5496688741721855 || TN : 0.49206349206349204
Score : 0.4657039711191336 || TP : 0.46308724832214765 || TN : 0.46875
Score : 0.5342960288808665 || TP : 0.5734265734265734 || TN : 0.4925373134328358
Score : 0.5270758122743683 || TP : 0.541095890410959 || TN : 0.5114503816793893
Score : 0.5054151624548736 || TP : 0.5657894736842105 || TN : 0.432
Score : 0.48014440433212996 || TP : 0.5266666666666666 || TN : 0.4251968503937008
Score : 0.5090252707581228 || TP : 0.5302013422818792 || TN : 0.484375
Score : 0.5126353790613718 || TP : 0.56 || TN : 0.4566929133858268
Score : 0.5451263537906137 || TP : 0.5933333333333334 || TN : 0.4881889763779528
Score : 0.4729241877256318 || TP : 0.5131578947368421 || TN : 0.424
Score : 0.534296028

In [27]:
print("MLP score moyen : {} || TP moyen : {} || TN moyen : {}".format(mean(scores),mean(TPs),mean(TNs)))

MLP score moyen : 0.5078852365570967 || TP moyen : 0.5443692036536304 || TN moyen : 0.4653102613033886


### Using TensorFlow

In [13]:
%load_ext tensorboard

In [14]:
!rm -rf ./logs/

In [15]:
shape = (len(predict_data.columns)-3,)
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=shape),
    tf.keras.layers.Dense(200, activation='relu'),
    tf.keras.layers.Dense(2000, activation='relu'),
    tf.keras.layers.Dense(2000, activation='relu'),
    tf.keras.layers.Dense(200, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [16]:
print(train_data.shape, test_data.shape)

(178341, 63) (44586, 63)


In [17]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy', 'Precision', 'Recall'])

In [18]:
import datetime
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [19]:
model.fit(train_data, train_labels, epochs=10, validation_data=(test_data,test_labels),callbacks=[tensorboard_callback])

Epoch 1/10
5574/5574 [==============================] - 421s 75ms/step - loss: 0.6776 - accuracy: 0.5739 - precision: 0.5736 - recall: 0.5719 - val_loss: 0.6664 - val_accuracy: 0.5938 - val_precision: 0.5993 - val_recall: 0.5643
Epoch 2/10
 273/5574 [>.............................] - ETA: 5:57 - loss: 0.6669 - accuracy: 0.5928 - precision: 0.5995 - recall: 0.5509

KeyboardInterrupt: 

In [30]:
time = str(datetime.date.today())
#model.save('./models/1m_200_2000_200_'+time)

INFO:tensorflow:Assets written to: ./models/1m_200_2000_200_2022-09-29\assets


In [32]:
loaded_model = tf.keras.models.load_model('./models/1m_200_2000_200_'+time)

In [33]:
test_loss, test_acc = loaded_model.evaluate(test_data,  test_labels, verbose=2)

#print('\nTest accuracy:', test_acc)

3344/3344 - 21s - loss: 3.0073 - accuracy: 0.6279 - precision: 0.6242 - recall: 0.6940 - 21s/epoch - 6ms/step


ValueError: too many values to unpack (expected 2)

In [50]:
test_labels.value_counts()

1    55000
0    52000
Name: Target, dtype: int64

In [24]:
probability_model = tf.keras.Sequential([model, 
                                         tf.keras.layers.Softmax()])



In [59]:
#preds = np.argmax(probability_model.predict(test_data),axis=1)
preds = np.where(model.predict(test_data)>0.5, 1, 0)

In [60]:
np.unique(preds, return_counts=True)

(array([0, 1]), array([45852, 61148], dtype=int64))

In [17]:
conf = confusion_matrix(test_labels,preds)
conf

NameError: name 'confusion_matrix' is not defined

In [62]:
TN = conf[0,0] / (conf[0,0] + conf[1,0])
TN

0.6329276803629067

In [63]:
TP = conf[1,1] / (conf[1,1] + conf[0,1])
TP

0.6242068424151239

# Predicting 2021 with 2019 and 2020

In [13]:
data_19_20 = predict_data.loc[(predict_data["Unix"] >= 1546300800) & (predict_data["Unix"] < 1609459200)]

In [14]:
data_19_20

,Unix,Date,Volume USD,Variation,RSI,MACD,MACD_H,-DM,+DM,ADX14,Variation-1,Vol-1,RSI-1,MACD-1,MACD_H-1,Variation-2,Vol-2,RSI-2,MACD-2,MACD_H-2,Variation-3,Vol-3,RSI-3,MACD-3,MACD_H-3,Variation-4,Vol-4,RSI-4,MACD-4,MACD_H-4,Variation-5,Vol-5,RSI-5,MACD-5,MACD_H-5,Variation-6,Vol-6,RSI-6,MACD-6,MACD_H-6,...,Vol-22,RSI-22,MACD-22,MACD_H-22,Variation-23,Vol-23,RSI-23,MACD-23,MACD_H-23,Variation-24,Vol-24,RSI-24,MACD-24,MACD_H-24,Variation-25,Vol-25,RSI-25,MACD-25,MACD_H-25,Variation-26,Vol-26,RSI-26,MACD-26,MACD_H-26,Variation-27,Vol-27,RSI-27,MACD-27,MACD_H-27,Variation-28,Vol-28,RSI-28,MACD-28,MACD_H-28,Variation-29,Vol-29,RSI-29,MACD-29,MACD_H-29,Target
1051110,1546302540,2019-01-01 00:29:00,-0.339529,-0.000855,-1.274979,-0.034384,-0.084242,-0.156757,-0.163178,2.124061,-0.000855,-0.348348,-1.604573,-0.040456,-0.083440,-0.024858,-0.348678,-1.530843,-0.045673,-0.080539,-0.000855,-0.345722,-1.530843,-0.049250,-0.073884,-0.170817,-0.346573,-1.809160,-0.057949,-0.082094,0.169270,-0.348678,-0.934083,-0.057667,-0.064931,-0.000855,-0.346812,-1.248113,-0.056799,-0.049652,...,-0.333742,-0.531060,-0.029239,-0.037302,0.110160,-0.320416,-0.190011,-0.031282,-0.035239,0.180212,-0.307819,0.112435,-0.025670,-0.013362,-0.025324,-0.346650,0.053628,-0.021920,-0.000780,0.095597,-0.343283,0.529031,-0.015081,0.017446,-0.275665,-0.331139,-0.094008,-0.019997,0.000966,0.192724,-0.175991,0.014096,-0.016292,0.010564,499.722633,-0.169705,3.246387,4.969239,13.123672,1
1051140,1546304340,2019-01-01 00:59:00,-0.310928,-0.026260,-0.057984,0.014677,0.000173,-0.264732,-0.163178,-0.404912,-0.058384,-0.104613,-0.272806,0.012934,-0.004468,0.166512,-0.325771,0.475583,0.017767,0.009196,0.139524,-0.345920,0.828898,0.026676,0.030900,-0.049207,-0.266947,1.158432,0.031503,0.037475,0.220858,-0.348678,1.875038,0.043348,0.061279,-0.000855,-0.348678,1.683387,0.052123,0.072213,...,-0.348678,-1.152155,0.006782,-0.038910,-0.000855,-0.272475,-1.152155,0.005821,-0.033668,-0.062648,-0.302550,-0.545211,0.002642,-0.035334,0.138586,-0.338360,-0.310238,0.005411,-0.020951,-0.069324,-0.250277,0.220905,0.004904,-0.018100,-0.148383,-0.265816,-0.474624,-0.001168,-0.030527,-0.246781,-0.114070,-1.163170,-0.015265,-0.061674,-0.000855,-0.306172,-1.274979,-0.026146,-0.078093,0
1051170,1546306140,2019-01-01 01:29:00,-0.348357,-0.317363,1.156960,0.084504,0.031952,-0.296745,-0.163178,1.617640,-0.092476,-0.348678,0.595365,0.073322,-0.003987,-0.000855,-0.348678,0.595365,0.063715,-0.028576,-0.000855,-0.308313,-0.445842,0.055453,-0.044696,0.100412,-0.328429,0.024384,0.052179,-0.044406,-0.144754,-0.173576,-0.457621,0.043546,-0.058338,0.053871,-0.348678,-0.257579,0.038352,-0.060396,...,-0.303221,-0.034856,0.013373,-0.045575,0.074453,-0.257242,0.291677,0.014419,-0.033696,-0.038253,-0.336800,0.285136,0.013643,-0.029007,-0.051689,-0.348678,-0.310548,0.010940,-0.030351,-0.000855,-0.348432,-0.522989,0.008686,-0.030235,-0.000855,-0.339206,-0.924383,0.006812,-0.029142,0.161765,-0.250235,0.112219,0.011429,-0.011113,-0.027694,-0.261352,0.047302,0.013886,-0.002398,1
1051200,1546307940,2019-01-01 01:59:00,-0.347857,-0.000855,0.264106,0.010963,-0.007443,-0.296745,-0.163178,0.186246,-0.000855,-0.328735,0.264106,0.010308,-0.007686,-0.000855,-0.330550,0.264106,0.009667,-0.007843,-0.157198,-0.348678,-0.613493,0.003125,-0.023560,-0.000855,-0.263270,-1.372702,-0.002046,-0.032513,-0.081262,-0.311185,-0.995764,-0.009125,-0.044718,0.235659,-0.348408,-0.240574,-0.005636,-0.026555,...,-0.348678,1.614297,0.065033,0.101136,-0.000855,-0.348678,1.614297,0.069342,0.092293,-0.000855,-0.324875,1.582944,0.071916,0.080638,0.069175,-0.348678,1.841273,0.075763,0.074675,-0.000855,-0.348678,2.549422,0.077903,0.065394,-0.000855,-0.348212,2.935006,0.078681,0.054373,0.234564,-0.348678,3.226769,0.087326,0.066343,-0.000855,-0.196373,3.218673,0.093094,0.068315,1
1051230,1546309740,2019-01-01 02:29:00,-0.348208,-0.138552,-0.338519,-0.008687,-0.000427,-0.246827,-0.163178,1.127477,-0.007601,0.393381,-0.355105,-0.010749,-0.005792,0.168274,-0.303581,

In [15]:
data_2021 = predict_data.loc[predict_data["Unix"] >= 1609459200]

In [16]:
data_2021

,Unix,Date,Volume USD,Variation,RSI,MACD,MACD_H,-DM,+DM,ADX14,Variation-1,Vol-1,RSI-1,MACD-1,MACD_H-1,Variation-2,Vol-2,RSI-2,MACD-2,MACD_H-2,Variation-3,Vol-3,RSI-3,MACD-3,MACD_H-3,Variation-4,Vol-4,RSI-4,MACD-4,MACD_H-4,Variation-5,Vol-5,RSI-5,MACD-5,MACD_H-5,Variation-6,Vol-6,RSI-6,MACD-6,MACD_H-6,...,Vol-22,RSI-22,MACD-22,MACD_H-22,Variation-23,Vol-23,RSI-23,MACD-23,MACD_H-23,Variation-24,Vol-24,RSI-24,MACD-24,MACD_H-24,Variation-25,Vol-25,RSI-25,MACD-25,MACD_H-25,Variation-26,Vol-26,RSI-26,MACD-26,MACD_H-26,Variation-27,Vol-27,RSI-27,MACD-27,MACD_H-27,Variation-28,Vol-28,RSI-28,MACD-28,MACD_H-28,Variation-29,Vol-29,RSI-29,MACD-29,MACD_H-29,Target
2064420,1609460280,2021-01-01 00:18:00,0.440705,-0.111944,-0.418507,-0.859043,0.170810,0.486209,-0.163178,0.387179,-0.061275,-0.287596,-0.363714,-0.821848,0.234938,0.028691,-0.183870,-0.515060,-0.774573,0.312876,-0.416947,-0.144609,-0.728193,-0.852356,0.044756,0.056217,1.563090,-0.551666,-0.886675,-0.054885,-0.146590,1.640388,-0.868890,-0.946685,-0.202488,0.541444,-0.123238,-0.116417,-0.822243,0.166855,...,0.743293,3.242280,142.804882,223.364022,-0.097894,1.468010,3.244261,152.254226,203.661526,0.080930,0.016519,3.244686,157.946879,177.972307,-0.171237,0.147181,3.243784,160.556058,149.272717,-0.092077,2.297348,3.243301,160.743352,119.913106,0.046214,-0.140457,3.243301,159.072290,91.514629,0.055845,0.048990,3.243431,155.967172,65.006292,-0.212085,-0.148241,3.242313,151.693925,40.712789,1
2064450,1609462080,2021-01-01 00:48:00,0.365838,-0.077123,-0.742743,-1.493196,0.504541,-0.296745,0.395642,1.314715,0.067571,3.039315,-0.085988,-1.423845,0.586897,-0.110258,0.339540,-0.610844,-1.385947,0.569664,-0.161734,1.869252,-1.072349,-1.388459,0.449094,0.238696,-0.127895,-0.923258,-1.303224,0.584510,-0.122605,-0.145849,-0.960070,-1.257936,0.587284,-0.037191,0.589914,-0.686703,-1.218965,0.572811,...,2.994560,-0.917234,-0.932375,0.032842,0.073664,-0.164294,-0.509659,-0.881114,0.161733,-0.154789,0.023824,-0.725001,-0.876723,0.140989,0.073330,-0.024904,-0.731635,-0.841248,0.206535,-0.184985,-0.105216,-1.236472,-0.858640,0.119269,-0.227717,0.736099,-1.454281,-0.929892,-0.092872,-0.151104,-0.082928,-1.319534,-1.019712,-0.311650,0.543953,-0.071406,-0.484481,-0.916955,0.022221,1
2064480,1609463880,2021-01-01 01:18:00,-0.046590,0.302298,-0.320223,-0.465584,-0.272004,-0.296745,-0.163178,0.126664,0.026329,1.157410,-0.081108,-0.424339,-0.108610,0.107621,2.662998,-0.704713,-0.354418,0.097881,0.211869,1.674685,-0.033680,-0.231284,0.403690,-0.147755,8.652587,0.125610,-0.176659,0.467301,-0.728076,1.365087,-0.493317,-0.351837,-0.089074,0.019034,1.739947,-0.323762,-0.479093,-0.407540,...,-0.111265,-1.324809,-1.874168,-0.193178,-0.080923,0.040390,-1.256137,-1.867260,-0.136287,0.041302,0.224345,-1.364099,-1.827994,-0.005268,0.098931,1.590074,-0.801049,-1.746615,0.210834,-0.004452,-0.083957,-1.014166,-1.664036,0.386885,-0.216778,-0.172972,-1.329838,-1.644890,0.360102,0.060913,0.432634,-0.948714,-1.592725,0.425931,-0.001648,-0.153209,-0.369843,-1.533950,0.496058,1
2064510,1609465680,2021-01-01 01:48:00,0.106845,-0.238532,-0.342232,0.187035,-0.616930,-0.127458,-0.163178,-1.019139,0.082811,0.754194,-0.864093,0.150736,-0.589467,-0.028349,0.819230,-0.832205,0.112245,-0.573286,-0.080704,-0.106752,-0.630703,0.056633,-0.605588,-0.078693,1.736232,-0.717821,-0.011141,-0.663567,0.233966,0.048825,-0.509649,0.006883,-0.483224,-0.026237,1.650204,0.117545,0.013230,-0.369806,...,0.909504,-0.603004,-0.303311,-0.806747,0.225206,1.987658,-0.379283,-0.341430,-0.746129,0.010958,-0.027614,-0.855193,-0.363846,-0.656136,-0.099089,0.559210,-0.764450,-0.406956,-0.638830,-0.088049,1.085145,-0.701146,-0.462439,-0.657680,0.102791,0.866412,-0.220169,-0.469361,-0.544436,0.155609,1.075887,-0.262514,-0.422186,-0.310888,-0.399651,2.024245,-0.538756,-0.500986,-0.456942,1
2064540,1609467480,2021-01-01 02:18:00,-0.089069,-0.210981,0.977215,0.548888,0.241416,0.877958,-0.163178,0.340086,-0.012847,0.292719,0.779643,0.513945,0.100795,-0.141577,0.896480,

In [17]:
split_mode = "STATIC"

#if split_mode == "RAND":
#    train_data, test_data, train_labels, test_labels = train_test_split(predict_data.drop(["Date", "Unix", "Target"],axis=1), predict_data["Target"], test_size=0.2, random_state=100)
if split_mode == "STATIC":
    train_data = data_19_20.drop(["Date", "Unix", "Target"],axis=1)
    train_labels = data_19_20["Target"]
    test_data = data_2021.drop(["Date", "Unix", "Target"],axis=1)
    test_labels = data_2021["Target"]

In [24]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [37]:
shape = (len(test_data.columns),)
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=shape),
    tf.keras.layers.Dense(200, activation='relu'),
    tf.keras.layers.Dense(2000, activation='relu'),
    tf.keras.layers.Dense(2000, activation='relu'),
    tf.keras.layers.Dense(200, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [38]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy', 'Precision', 'Recall', 'AUC'])

In [39]:
import datetime
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [40]:
model.fit(train_data, train_labels, epochs=10, validation_data=(test_data,test_labels),callbacks=[tensorboard_callback], batch_size=32)

Epoch 1/10
1056/1056 [==============================] - 51s 43ms/step - loss: 0.2498 - accuracy: 0.9039 - precision: 0.9080 - recall: 0.9008 - auc: 0.9679 - val_loss: 0.3681 - val_accuracy: 0.8939 - val_precision: 0.9067 - val_recall: 0.8786 - val_auc: 0.9573
Epoch 2/10
1056/1056 [==============================] - 40s 38ms/step - loss: 0.1964 - accuracy: 0.9244 - precision: 0.9270 - recall: 0.9226 - auc: 0.9797 - val_loss: 0.3422 - val_accuracy: 0.8857 - val_precision: 0.8885 - val_recall: 0.8826 - val_auc: 0.9520
Epoch 3/10
1056/1056 [==============================] - 41s 39ms/step - loss: 0.1716 - accuracy: 0.9310 - precision: 0.9327 - recall: 0.9304 - auc: 0.9832 - val_loss: 0.2633 - val_accuracy: 0.8977 - val_precision: 0.9104 - val_recall: 0.8827 - val_auc: 0.9639
Epoch 4/10
1056/1056 [==============================] - 41s 39ms/step - loss: 0.1554 - accuracy: 0.9360 - precision: 0.9375 - recall: 0.9354 - auc: 0.9860 - val_loss: 0.2870 - val_accuracy: 0.8973 - val_precision: 0.8982

In [36]:
test_loss, test_acc, test_prec, test_rec, test_auc = model.evaluate(test_data,  test_labels, verbose=2)

#print('\nTest accuracy:', test_acc)

636/636 - 4s - loss: 0.5183 - accuracy: 0.8707 - precision: 0.9138 - recall: 0.8192 - auc: 0.9448 - 4s/epoch - 7ms/step


ValueError: too many values to unpack (expected 2)

In [25]:
time = str(datetime.date.today())
model.save('./models/30_to_10m_200_2000_200_y2019_2020_'+time)

INFO:tensorflow:Assets written to: ./models/20_to_6m_200_2000_200_y2019_2020_2022-10-05\assets
